In [8]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
from io import BytesIO
import base64
from pathlib import Path
import pandas as pd
import numpy as np
from transformers import ViTImageProcessor, ViTModel, ViTFeatureExtractor
from PIL import Image
import torch
import requests
import tqdm as tqdm
from sentence_transformers import SentenceTransformer, util
from PIL import Image, ImageFile
import requests
import torch

In [10]:
BATCH_SIZE = 64

In [11]:
df = pd.read_csv('/kaggle/input/nto-cv-olympiad-final-dataset/merged/merged.csv')

In [12]:
def img_from_base64(data: str) -> Image.Image:
    return Image.open(BytesIO(base64.b64decode(data, validate=False)))

In [13]:
images = list(df.image.apply(img_from_base64))

In [15]:
# We use the original clip-ViT-B-32 for encoding images
img_model = SentenceTransformer('clip-ViT-B-32')

# Our text embedding model is aligned to the img_model and maps 50+
# languages to the same vector space
text_model = SentenceTransformer('sentence-transformers/clip-ViT-B-32-multilingual-v1')

# Map images to the vector space
# img_embeddings = img_model.encode(images[:10])

# # Now we encode our text:
# texts = [
#     "A dog in the snow",
#     "Eine Katze",  # German: A cat
#     "Una playa con palmeras."  # Spanish: a beach with palm trees
# ]

# text_embeddings = text_model.encode(texts)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [17]:
img_model.save('img_model')
text_model.save('text_model')

In [20]:
img_model.load('img_model')
text_model.load('text_model')

SentenceTransformer(
  (0): CLIPModel()
)

In [ ]:
img_features = img_model.encode(images, batch_size=128)

In [ ]:
place_df = pd.read_csv('/kaggle/input/nto-cv-olympiad-final-dataset/descriptions.csv')
place_df = place_df.loc[place_df.description != '\t'].reset_index(drop=True)

In [ ]:
text_features = text_model.encode(place_df.description)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
pca.fit(img_features)

In [ ]:
data = pca.transform(img_features)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
xids = ['U7274', 'N3937686397', 'W34939464','N4995217430']
subset = df.loc[df.XID.apply(lambda xid: xid in xids)]

In [ ]:
data = pca.transform(img_features[subset.index])

In [ ]:
sns.scatterplot(x=data[:, 0], y=data[:, 1], hue=subset.XID, sizes=(.1, .1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Embeddings of different places, projected into 2D')
plt.savefig('plemb.png', bbox_inches='tight')

In [ ]:
text_data = pca.transform(text_features)

In [ ]:
sns.scatterplot(x=data[:, 0], y=data[:, 1])

In [ ]:
def cs(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [ ]:
cs(text_features[0], text_features[1])

In [ ]:
norms = np.linalg.norm(text_features, axis=-1)
sim = (text_features @ text_features.T) / (norms[None, ...] * norms[..., None])

In [ ]:
sim *= (1 - np.eye(len(sim)))

In [ ]:
sns.heatmap(sim)

In [ ]:
text_emb = text_features[100][None]

In [ ]:
place_df.iloc[100]

In [ ]:
sims = ((img_features @ text_emb.T)[:, 0] / np.linalg.norm(text_emb) / np.linalg.norm(img_features, axis=-1))

In [ ]:
sims.argsort()[::-1][:5]

In [ ]:
tdf = df.iloc[sims.argsort()[::-1][:100]].reset_index()
tdf.loc[tdf.XID == 'W38411380']